In [1]:
import numpy as np
import pandas as pd
import f90nml
import os
from collections import OrderedDict

need to convert 6e-07 to $6\times10^{-07}$ everywhere

In [2]:
#nml=f90nml.read('namelist_smelt_ref')
#nmlold=f90nml.read('namelist_smelt_ref_old') # for now just load the same one
#nml1_11=f90nml.read('namelist_smelt_cfg_1_11')
nml=f90nml.read('namelist_smelt_cfg_HC201905equiv')
nmlold=f90nml.read('namelist_smelt_cfg_HC201812equiv')
#nml1_11=nml # 11% correction was already done in these namelists

In [3]:
print(nml['nampismezo']['zz_rate_mesozoo_sumpeakval'][0]*.5)

0.275


In [4]:
nml['nampismezo']['zz_rate_mesozoo_sumpeakval'][0]*nml['nampismezo']['zz_rate_mesozoo_r']

8.363850000000001e-06

In [5]:
# Remove irrelevant values (that are only irrelevant for either new or old nml):
for iii in ('zz_frac_waste_MNM_NH','zz_frac_waste_MNM_DON','zz_frac_waste_MNM_PON',
           'zz_frac_waste_MEX_NH','zz_frac_waste_MEX_DON','zz_frac_waste_MEX_PON'):
    nml['nampismezo'][iii]='-'
    nmlold['nampismezo'][iii]='-'
    #nml1_11['nampismezo'][iii]='-'
for iii in ('zzigamma','zzisigma'):
    nml['nampisopt'][iii]='-'    
    #nml1_11['nampisopt'][iii]='-'    
#for iii in ('zz_alpha_b_Si','zz_alpha_b_N'):
#    nmlold['nampissink'][iii]='-' 
nmlold['nampissink']['zz_alpha_b_D']=nmlold['nampissink']['zz_alpha_b_Si']
# rescale optical model parameters for 1.6 chl:N instead of 2.0:
nmlold['nampisopt']['zzibeta']=np.round(nml['nampisopt']['zzibeta']*(2.0/1.6)**0.665,4)
nmlold['nampisopt']['zzN2chl']=1.6
# add some additional information:
#nmlold['nampisopt']['zzitheta']=str(nmlold['nampisopt']['zzitheta'])+r'$+f(Q)$ \textsuperscript{a}'
nml['nampissink']['zz_w_sink_Pmicro_min']="{:.2e}".format(1/3600/24*nml['nampissink']['zz_w_sink_Pmicro_min'])+' ['+"{:.1f}".format(nml['nampissink']['zz_w_sink_Pmicro_min'])+']'
nml['nampissink']['zz_w_sink_Pmicro_max']="{:.2e}".format(1/3600/24*nml['nampissink']['zz_w_sink_Pmicro_max'])+' ['+"{:.1f}".format(nml['nampissink']['zz_w_sink_Pmicro_max'])+']'
nml['nampissink']['zz_w_sink_D_PON']="{:.2e}".format(nml['nampissink']['zz_w_sink_D_PON'])+' ['+"{:.1f}".format(3600*24*nml['nampissink']['zz_w_sink_D_PON'])+']'
nml['nampissink']['zz_w_sink_D_bSi']="{:.2e}".format(nml['nampissink']['zz_w_sink_D_bSi'])+' ['+"{:.1f}".format(3600*24*nml['nampissink']['zz_w_sink_D_bSi'])+']'
nmlold['nampissink']['zz_w_sink_Pmicro_min']="{:.2e}".format(1/3600/24*nmlold['nampissink']['zz_w_sink_Pmicro_min'])+' ['+"{:.1f}".format(nmlold['nampissink']['zz_w_sink_Pmicro_min'])+']'
nmlold['nampissink']['zz_w_sink_Pmicro_max']="{:.2e}".format(1/3600/24*nmlold['nampissink']['zz_w_sink_Pmicro_max'])+' ['+"{:.1f}".format(nmlold['nampissink']['zz_w_sink_Pmicro_max'])+']'
nmlold['nampissink']['zz_w_sink_D_PON']="{:.2e}".format(nmlold['nampissink']['zz_w_sink_D_PON'])+' ['+"{:.1f}".format(3600*24*nmlold['nampissink']['zz_w_sink_D_PON'])+']'
nmlold['nampissink']['zz_w_sink_D_bSi']="{:.2e}".format(nmlold['nampissink']['zz_w_sink_D_bSi'])+' ['+"{:.1f}".format(3600*24*nmlold['nampissink']['zz_w_sink_D_bSi'])+']'

#nml1_11['nampissink']['zz_w_sink_Pmicro_min']="{:.2e}".format(1/3600/24*nml1_11['nampissink']['zz_w_sink_Pmicro_min'])+' ['+"{:.3f}".format(nml1_11['nampissink']['zz_w_sink_Pmicro_min'])+']'
#nml1_11['nampissink']['zz_w_sink_Pmicro_max']="{:.2e}".format(1/3600/24*nml1_11['nampissink']['zz_w_sink_Pmicro_max'])+' ['+"{:.3f}".format(nml1_11['nampissink']['zz_w_sink_Pmicro_max'])+']'
#nml1_11['nampissink']['zz_w_sink_D_PON']="{:.2e}".format(nml1_11['nampissink']['zz_w_sink_D_PON'])+' ['+"{:.1f}".format(3600*24*nml1_11['nampissink']['zz_w_sink_D_PON'])+']'
#nml1_11['nampissink']['zz_w_sink_D_bSi']="{:.2e}".format(nml1_11['nampissink']['zz_w_sink_D_bSi'])+' ['+"{:.1f}".format(3600*24*nml1_11['nampissink']['zz_w_sink_D_bSi'])+']'

# change nampismezo concentrations to rates:
nml['nampismezo']['zz_rate_mesozoo_winterconc']="{:.3g}".format(nml['nampismezo']['zz_rate_mesozoo_winterconc']*nml['nampismezo']['zz_rate_mesozoo_r'])
nml['nampismezo']['zz_rate_mesozoo_sumpeakval']="[{0:.3g}, {1:.3g}, {2:.3g}]".format(
            nml['nampismezo']['zz_rate_mesozoo_sumpeakval'][0]*nml['nampismezo']['zz_rate_mesozoo_r'],
            nml['nampismezo']['zz_rate_mesozoo_sumpeakval'][1]*nml['nampismezo']['zz_rate_mesozoo_r'],
            nml['nampismezo']['zz_rate_mesozoo_sumpeakval'][2]*nml['nampismezo']['zz_rate_mesozoo_r'])
nmlold['nampismezo']['zz_rate_mesozoo_winterconc']="{:.3g}".format(nmlold['nampismezo']['zz_rate_mesozoo_winterconc']*nmlold['nampismezo']['zz_rate_mesozoo_r'])
nmlold['nampismezo']['zz_rate_mesozoo_sumpeakval']="[{0:.3g}, {1:.3g}, {2:.3g}]".format(
            nmlold['nampismezo']['zz_rate_mesozoo_sumpeakval'][0]*nmlold['nampismezo']['zz_rate_mesozoo_r'],
            nmlold['nampismezo']['zz_rate_mesozoo_sumpeakval'][1]*nmlold['nampismezo']['zz_rate_mesozoo_r'],
            nmlold['nampismezo']['zz_rate_mesozoo_sumpeakval'][2]*nmlold['nampismezo']['zz_rate_mesozoo_r'])
#nml1_11['nampismezo']['zz_rate_mesozoo_winterconc']="{:.3g}".format(nml1_11['nampismezo']['zz_rate_mesozoo_winterconc']*nml1_11['nampismezo']['zz_rate_mesozoo_r'])
#nml1_11['nampismezo']['zz_rate_mesozoo_sumpeakval']="[{0:.3g}, {1:.3g}, {2:.3g}]".format(
#            nml1_11['nampismezo']['zz_rate_mesozoo_sumpeakval'][0]*nml1_11['nampismezo']['zz_rate_mesozoo_r'],
#            nml1_11['nampismezo']['zz_rate_mesozoo_sumpeakval'][1]*nml1_11['nampismezo']['zz_rate_mesozoo_r'],
#            nml1_11['nampismezo']['zz_rate_mesozoo_sumpeakval'][2]*nml1_11['nampismezo']['zz_rate_mesozoo_r'])
print(nml['nampismezo']['zz_rate_mesozoo_sumpeakval'])
# this must be last:
nmlold['nampismezo']['zz_rate_mesozoo_r']='-'

[8.36e-06, 8.36e-06, 5.47e-06]


In [6]:
nml['nampissink']['zz_w_sink_Pmicro_min']

'6.42e-06 [0.6]'

In [7]:
nml.keys()

odict_keys(['nampisprod', 'nampismort', 'nampismes', 'nampismezo', 'nampiszoo', 'nampisrem', 'nampisopt', 'nampissink', 'nampisriv', 'nampissbc', 'nampiscarbon', 'nampisext'])

In [8]:
nml['nampisprod'].keys()

odict_keys(['zz_rate_r_diat', 'zz_rate_r_myri', 'zz_rate_r_nano', 'zz_rate_maxtemp_diat', 'zz_rate_maxtemp_myri', 'zz_rate_maxtemp_nano', 'zz_rate_temprange_diat', 'zz_rate_temprange_myri', 'zz_rate_temprange_nano', 'zz_rate_iopt_diat', 'zz_rate_iopt_myri', 'zz_rate_iopt_nano', 'zz_rate_gamma_diat', 'zz_rate_gamma_myri', 'zz_rate_gamma_nano', 'zz_rate_k_si_diat', 'zz_rate_k_si_myri', 'zz_rate_k_si_nano', 'zz_rate_kapa_diat', 'zz_rate_kapa_myri', 'zz_rate_kapa_nano', 'zz_rate_k_diat', 'zz_rate_k_myri', 'zz_rate_k_nano', 'zz_rate_si_ratio_diat', 'zz_rate_si_ratio_myri', 'zz_rate_si_ratio_nano'])

In [9]:
nml['nampismes'].keys()

odict_keys(['zz_rate_mesorub_r', 'zz_rate_mesorub_picopredslope', 'zz_rate_mesorub_picohalfsat', 'zz_rate_mesorub_eff', 'zz_frac_waste_fen_nh', 'zz_frac_waste_fen_don', 'zz_frac_waste_fen_pon', 'zz_frac_waste_fen_bsi'])

In [10]:
nml['nampiszoo'].keys()

odict_keys(['zz_rate_uzoo_rm', 'zz_rate_uzoo_excr', 'zz_rate_uzoo_predslope', 'zz_rate_uzoo_halfsat', 'zz_rate_uzoo_micropref', 'zz_rate_uzoo_micropredslope', 'zz_rate_uzoo_microhalfsat', 'zz_rate_uzoo_nanopref', 'zz_rate_uzoo_nanopredslope', 'zz_rate_uzoo_nanohalfsat', 'zz_rate_uzoo_picopref', 'zz_rate_uzoo_picopredslope', 'zz_rate_uzoo_picohalfsat', 'zz_rate_uzoo_pon_pref', 'zz_rate_uzoo_pon_predslope', 'zz_rate_uzoo_pon_halfsat', 'zz_rate_uzoo_z_pref', 'zz_rate_uzoo_z_predslope', 'zz_rate_uzoo_z_halfsat', 'zz_rate_uzoo_r', 'zz_rate_uzoo_eff', 'zz_frac_waste_pez_nh', 'zz_frac_waste_pez_don', 'zz_frac_waste_pez_pon', 'zz_frac_waste_dez_nh', 'zz_frac_waste_dez_don', 'zz_frac_waste_dez_pon', 'zz_frac_waste_dez_bsi', 'zz_frac_waste_nez_nh', 'zz_frac_waste_nez_don', 'zz_frac_waste_nez_pon', 'zz_frac_waste_nez_bsi', 'zz_frac_waste_fez_nh', 'zz_frac_waste_fez_don', 'zz_frac_waste_fez_pon', 'zz_frac_waste_fez_bsi', 'zz_frac_waste_zez_nh', 'zz_frac_waste_zez_don', 'zz_frac_waste_zez_pon', 'zz

In [11]:
nml['nampismezo'].keys()

odict_keys(['zz_rate_mesozoo_winterconc', 'zz_rate_mesozoo_sumpeakval', 'zz_rate_mesozoo_sumpeakwid', 'zz_rate_mesozoo_sumpeakpos', 'zz_rate_mesozoo_alpha', 'zz_rate_mesozoo_r', 'zz_rate_mesozoo_rm', 'zz_rate_mesozoo_excr', 'zz_rate_mesozoo_predslope', 'zz_rate_mesozoo_halfsat', 'zz_rate_mesozoo_micropref', 'zz_rate_mesozoo_micropredslope', 'zz_rate_mesozoo_microhalfsat', 'zz_rate_mesozoo_nanopref', 'zz_rate_mesozoo_nanopredslope', 'zz_rate_mesozoo_nanohalfsat', 'zz_rate_mesozoo_picopref', 'zz_rate_mesozoo_picopredslope', 'zz_rate_mesozoo_picohalfsat', 'zz_rate_mesozoo_pon_pref', 'zz_rate_mesozoo_pon_predslope', 'zz_rate_mesozoo_pon_halfsat', 'zz_rate_mesozoo_z_pref', 'zz_rate_mesozoo_z_predslope', 'zz_rate_mesozoo_z_halfsat', 'zz_rate_mesozoo_eff', 'zz_frac_waste_mnm_nh', 'zz_frac_waste_mnm_don', 'zz_frac_waste_mnm_pon', 'zz_frac_waste_mex_nh', 'zz_frac_waste_mex_don', 'zz_frac_waste_mex_pon', 'zz_frac_waste_pem_nh', 'zz_frac_waste_pem_don', 'zz_frac_waste_pem_pon', 'zz_frac_waste_dem

In [12]:
matchDict=dict()
# format, codevarname, (textvarname, namelistheading, units)
matchDict['Photosynthetic Growth']= OrderedDict([
('zz_rate_r_DIAT',('\mu^{DIAT}_{max}','nampisprod','s$^{-1}$')),
('zz_rate_r_myri',('\mu^{MRUB}_{max}','nampisprod','s$^{-1}$')),
('zz_rate_r_nano',('\mu^{FLAG}_{max}','nampisprod','s$^{-1}$')),
('zz_rate_maxtemp_DIAT',('\Theta_{max}^{DIAT}','nampisprod','$^{\circ}$C')),
('zz_rate_maxtemp_myri',('\Theta_{max}^{MRUB}','nampisprod','$^{\circ}$C')),
('zz_rate_maxtemp_nano',('\Theta_{max}^{FLAG}','nampisprod','$^{\circ}$C')),
('zz_rate_temprange_DIAT',('\Theta_{range}^{DIAT}','nampisprod','$^{\circ}$C')),
('zz_rate_temprange_myri',('\Theta_{range}^{MRUB}','nampisprod','$^{\circ}$C')),
('zz_rate_temprange_nano',('\Theta_{range}^{FLAG}','nampisprod','$^{\circ}$C')),
('zz_rate_iopt_DIAT',(r'I_{\text{opt}}^{DIAT}','nampisprod','W m$^{-2}$')),
('zz_rate_iopt_myri',(r'I_{\text{opt}}^{MRUB}','nampisprod','W m$^{-2}$')),
('zz_rate_iopt_nano',(r'I_{\text{opt}}^{FLAG}','nampisprod','W m$^{-2}$')),
('zz_rate_k_si_DIAT',('k_\mathrm{Si}^{DIAT}','nampisprod',r'$\upmu$M Si')),
('zz_rate_k_si_myri',('k_\mathrm{Si}^{MRUB}','nampisprod',r'$\upmu$M Si')),
('zz_rate_k_si_nano',('k_\mathrm{Si}^{FLAG}','nampisprod',r'$\upmu$M Si')),
('zz_rate_kapa_DIAT',('\kappa^{DIAT}','nampisprod','')),
('zz_rate_kapa_myri',('\kappa^{MRUB}','nampisprod','')),
('zz_rate_kapa_nano',('\kappa^{FLAG}','nampisprod','')),
('zz_rate_k_DIAT',('K_\mathrm{N}^{DIAT}','nampisprod',r'$\upmu$M N')),
('zz_rate_k_myri',('K_\mathrm{N}^{MRUB}','nampisprod',r'$\upmu$M N')),
('zz_rate_k_nano',('K_\mathrm{N}^{FLAG}','nampisprod',r'$\upmu$M N')),
('zz_rate_si_ratio_DIAT',('a_{\mathrm{Si:N}}^{DIAT}','nampisprod',r'$\upmu$M Si ($\upmu$M N)$^{-1}$')),
('zz_rate_si_ratio_myri',('a_{\mathrm{Si:N}}^{MRUB}','nampisprod',r'$\upmu$M Si ($\upmu$M N)$^{-1}$')),
('zz_rate_si_ratio_nano',('a_{\mathrm{Si:N}}^{FLAG}','nampisprod',r'$\upmu$M Si ($\upmu$M N)$^{-1}$')) ])

# unused,
#'zz_rate_gamma_DIAT',('','nampisprod',''),
#'zz_rate_gamma_myri',('','nampisprod',''),
#'zz_rate_gamma_nano',('','nampisprod',''),

In [13]:
# format, codevarname, (textvarname, namelistheading, units)
matchDict['Grazing'] = OrderedDict([
('zz_rate_uzoo_r',(r'\upsilon_{max}^{MICZ}','nampiszoo','s$^{-1}$')), 
('zz_rate_uzoo_predslope',(r'\alpha^{MICZ}','nampiszoo',r'$\upmu$M N')), 
('zz_rate_uzoo_halfsat',(r'K^{MICZ}','nampiszoo',r'$\upmu$M N')), 
('zz_rate_uzoo_micropref',(r'\rho_{DIAT}^{MICZ}','nampiszoo','')), 
('zz_rate_uzoo_nanopref',(r'\rho_{MRUB}^{MICZ}','nampiszoo','')), 
('zz_rate_uzoo_picopref',(r'\rho_{FLAG}^{MICZ}','nampiszoo','')), 
('zz_rate_uzoo_pon_pref',(r'\rho_{PON}^{MICZ}','nampiszoo','')), 
('zz_rate_uzoo_z_pref',(r'\rho_{MICZ}^{MICZ}','nampiszoo','')), 
('zz_rate_uzoo_micropredslope',(r'\alpha_{DIAT}^{MICZ}','nampiszoo',r'$\upmu$M N')), 
('zz_rate_uzoo_nanopredslope',(r'\alpha_{MRUB}^{MICZ}','nampiszoo',r'$\upmu$M N')), 
('zz_rate_uzoo_picopredslope',(r'\alpha_{FLAG}^{MICZ}','nampiszoo',r'$\upmu$M N')), 
('zz_rate_uzoo_pon_predslope',(r'\alpha_{PON}^{MICZ}','nampiszoo',r'$\upmu$M N')), 
('zz_rate_uzoo_z_predslope',(r'\alpha_{MICZ}^{MICZ}','nampiszoo',r'$\upmu$M N')), 
('zz_rate_uzoo_microhalfsat',(r'K_{DIAT}^{MICZ}','nampiszoo',r'$\upmu$M N')), 
('zz_rate_uzoo_nanohalfsat',(r'K_{MRUB}^{MICZ}','nampiszoo',r'$\upmu$M N')), 
('zz_rate_uzoo_picohalfsat',(r'K_{FLAG}^{MICZ}','nampiszoo',r'$\upmu$M N')), 
('zz_rate_uzoo_pon_halfsat',(r'K_{PON}^{MICZ}','nampiszoo',r'$\upmu$M N')), 
('zz_rate_uzoo_z_halfsat',(r'K_{MICZ}^{MICZ}','nampiszoo',r'$\upmu$M N')), 
('zz_rate_uZoo_eff',(r'\epsilon^{MICZ}','nampiszoo','')), 
('zz_rate_mesozoo_r',(r'\upsilon_{max}^{MESZ}','nampismezo','s$^{-1}$')), 
('zz_rate_mesozoo_predslope',(r'\alpha^{MESZ}','nampismezo',r'$\upmu$M N')), 
('zz_rate_mesozoo_halfsat',(r'K^{MESZ}','nampismezo',r'$\upmu$M N')), 
('zz_rate_mesozoo_micropref',(r'\rho_{DIAT}^{MESZ}','nampismezo','')), 
('zz_rate_mesozoo_nanopref',(r'\rho_{MRUB}^{MESZ}','nampismezo','')), 
('zz_rate_mesozoo_picopref',(r'\rho_{FLAG}^{MESZ}','nampismezo','')), 
('zz_rate_mesozoo_pon_pref',(r'\rho_{PON}^{MESZ}','nampismezo','')), 
('zz_rate_mesozoo_z_pref',(r'\rho_{MICZ}^{MESZ}','nampismezo','')), 
('zz_rate_mesozoo_micropredslope',(r'\alpha_{DIAT}^{MESZ}','nampismezo',r'$\upmu$M N')), 
('zz_rate_mesozoo_nanopredslope',(r'\alpha_{MRUB}^{MESZ}','nampismezo',r'$\upmu$M N')), 
('zz_rate_mesozoo_picopredslope',(r'\alpha_{FLAG}^{MESZ}','nampismezo',r'$\upmu$M N')), 
('zz_rate_mesozoo_pon_predslope',(r'\alpha_{PON}^{MESZ}','nampismezo',r'$\upmu$M N')), 
('zz_rate_mesozoo_z_predslope',(r'\alpha_{MICZ}^{MESZ}','nampismezo',r'$\upmu$M N')), 
('zz_rate_mesozoo_microhalfsat',(r'K_{DIAT}^{MESZ}','nampismezo',r'$\upmu$M N')), 
('zz_rate_mesozoo_nanohalfsat',(r'K_{MRUB}^{MESZ}','nampismezo',r'$\upmu$M N')), 
('zz_rate_mesozoo_picohalfsat',(r'K_{FLAG}^{MESZ}','nampismezo',r'$\upmu$M N')), 
('zz_rate_mesozoo_pon_halfsat',(r'K_{PON}^{MESZ}','nampismezo',r'$\upmu$M N')), 
('zz_rate_mesozoo_z_halfsat',(r'K_{MICZ}^{MESZ}','nampismezo',r'$\upmu$M N')), 
('zz_rate_mesozoo_eff',(r'\epsilon^{MESZ}','nampismezo','')), 
('zz_rate_uzoo_Rm',(r'm_{MICZ}','nampiszoo','s$^{-1}$')),
('zz_rate_mesozoo_Rm',(r'm_{MESZ}','nampismezo','s$^{-1}$')),
('zz_rate_uzoo_excr',(r'e_{MICZ}','nampiszoo','s$^{-1}$')),
('zz_rate_mesozoo_excr',(r'e_{MESZ}','nampismezo','s$^{-1}$'))])


In [14]:
# make table for paper
ikey='Grazing'
lines=list()
lines.append(r'   \small'+'\n')
lines.append(r'   \centering'+'\n')
lines.append(r'   \begin{tabular}{lrrl}\toprule'+'\n')
lines.append(r'    Parameter & Current Value & Previous Value & Units \\ \midrule'+'\n')
for ivar in matchDict[ikey]:
    #if not ivar=='zz_rate_mesozoo_r':
    iltx=matchDict[ikey][ivar][0]
    ival=nml[matchDict[ikey][ivar][1]][ivar]
    #ival1_11=nml1_11[matchDict[ikey][ivar][1]][ivar]
    ivalold=nmlold[matchDict[ikey][ivar][1]][ivar]
    iunits=matchDict[ikey][ivar][2]
    if ival==ivalold:
        lines.append(r'     ${}$ & {} & {} & {} \\'.format(iltx,ival,' ',iunits)+'\n')
    else:
        lines.append(r'     ${}$ & {} & {} & {} \\'.format(iltx,ival,ivalold,iunits)+'\n')
    #else:
    #    lines.append(r'     ${}$ & {}* & {} & {} \\'.format(iltx,ival1_11,ivalold,iunits)+'\n')
lines.append(r'   \midrule'+'\n')
lines.append(r' \\ \\')
lines.append(r'   \end{tabular}'+'\n')

f = open('GrazingParamsTBL.tex', 'w')
f.writelines(lines)
f.close()

In [15]:
matchDict['Mesozooplankton Abundance'] = OrderedDict([
('zz_rate_mesozoo_winterconc',(r'\Upsilon_{w} ','nampismezo',r'$\upmu$M N s$^{-1}$')), 
('zz_rate_mesozoo_sumpeakval',(r'[\Upsilon_{1},\Upsilon_{2},\Upsilon_{3}]','nampismezo',r'$\upmu$M N s$^{-1}$')), 
('zz_rate_mesozoo_sumpeakwid',(r'[\sigma _1,\sigma _2,\sigma _3]','nampismezo','days')), 
('zz_rate_mesozoo_sumpeakpos',(r'[t_{01},t_{02},t_{03}]','nampismezo','year day'))])

In [16]:
# make table for paper 
# replace with rates by multiplying by zz_rate_mesozoo_r
ikey='Mesozooplankton Abundance'
lines=list()
lines.append(r'   \small'+'\n')
lines.append(r'   \centering'+'\n')
lines.append(r'   \begin{tabular}{lrrl}\toprule'+'\n')
lines.append(r'    Parameter & Current Value & Previous Value & Units \\ \midrule'+'\n')
for ivar in matchDict[ikey]:
    iltx=matchDict[ikey][ivar][0]
    ival=nml[matchDict[ikey][ivar][1]][ivar]
    #ival1_11=nml1_11[matchDict[ikey][ivar][1]][ivar]
    ivalold=nmlold[matchDict[ikey][ivar][1]][ivar]
    iunits=matchDict[ikey][ivar][2]
    #if ival==ival1_11:
    if ival==ivalold:
        lines.append(r'     ${}$ & {} & {} & {} \\'.format(iltx,ival,' ',iunits)+'\n')
    else:
        lines.append(r'     ${}$ & {} & {} & {} \\'.format(iltx,ival,ivalold,iunits)+'\n')
    #else:
    #    lines.append(r'     ${}$ & {}* & {} & {} \\'.format(iltx,ival1_11,ivalold,iunits)+'\n')
lines.append(r'   \midrule'+'\n')
lines.append(r' \\ \\')
lines.append(r'   \end{tabular}'+'\n')

f = open('MesozoTBL.tex', 'w')
f.writelines(lines)
f.close()

In [17]:
matchDict['Mortality'] = OrderedDict([
('zz_rate_micro_Rm',(r'm_{DIAT}','nampismort','s$^{-1}$')), 
('zz_rate_myri_Rm',(r'm_{MRUB}','nampismort','s$^{-1}$')), 
('zz_rate_pico_Rm',(r'm_{FLAG}','nampismort','s$^{-1}$'))])

In [18]:
matchDict['Mesodinium rubrum Grazing'] = OrderedDict([
('zz_rate_mesorub_r',(r'\upsilon_{max}^{MRUB}','nampismes','s$^{-1}$')), 
('zz_rate_mesorub_picopredslope',(r'\alpha^{MRUB}','nampismes',r'$\upmu$M N')), 
('zz_rate_mesorub_picohalfsat',(r'K^{MRUB}','nampismes',r'$\upmu$M N')), 
('zz_rate_mesorub_eff',(r'\epsilon^{MRUB}','nampismes',''))])

In [19]:
matchDict['Remineralization'] = OrderedDict([
('zz_remin_NH',(r'b_{\textrm{NH}}^{\textrm{NO}}','nampisrem',r's$^{-1}$ ($\upmu$M N)$^{-1}$')), 
('zz_remin_D_DON',(r'b_{\textrm{DON}}^{\textrm{NH}}','nampisrem','s$^{-1}$')), 
('zz_remin_D_PON',(r'b_{\textrm{PON}}^{\textrm{NH}}','nampisrem','s$^{-1}$')), 
('zz_remin_D_bSi',(r'b_{\textrm{bSi}}^{\textrm{dSi}}','nampisrem','s$^{-1}$'))])

In [20]:
matchDict['Light'] = OrderedDict([
('zzialpha',(r'\gamma','nampisopt','m$^{-1}$')), 
('zzibeta',(r'\beta','nampisopt','m$^{-1}$')), 
('zzitheta',(r'\lambda','nampisopt','m$^{-1}$')), 
('zzidl',(r'\delta','nampisopt','m')), 
('zzN2chl',(r'a_{\textrm{Chl:N}}','nampisopt','g Chl (mol N)$^{-1}$'))])

# note: in 201812 runs chl:N was set to 2.0, but in analysis 1.6 was used, 
# so numbers in namelist for alpha have to be rescaled
# need to solve betaRun*(2.0N)^0.665=betaTrue*(1.6N)^.665;
#   betaTrue=betaRun*(2.0/1.6)^.665; 
# these are changed in cell 3

In [21]:
np.round(0.0433*(2.0/1.6)**.665,4),nml['nampisopt']['zzibeta']

(0.0502, 0.0433)

In [22]:
matchDict['Sinking and Bottom Boundary Condition'] = OrderedDict([
('zz_w_sink_Pmicro_min',(r'w_{s\mathrm{min}}^{DIAT}','nampissink','m s$^{-1}$ [m d$^{-1}$]')), 
('zz_w_sink_Pmicro_max',(r'w_{s\mathrm{max}}^{DIAT}','nampissink','m s$^{-1}$ [m d$^{-1}$]')), 
('zz_w_sink_D_PON',(r'w_{s}^{PON}','nampissink','m s$^{-1}$ [m d$^{-1}$]')), 
('zz_w_sink_D_bSi',(r'w_{s}^{bSi}','nampissink','m s$^{-1}$ [m d$^{-1}$]')), 
('zz_alpha_b_Si',(r'\alpha_b^{Si}','nampissink','')), 
('zz_alpha_b_N',(r'\alpha_b^{N}','nampissink','')), 
('zz_alpha_b_D',(r'\alpha_b^{DIAT}','nampissink',''))])

In [23]:
matchDict['Detrital Transfer Coefficients'] = OrderedDict([
('zz_frac_waste_FEN_NH', (r'\chi_{FLAG\rightarrow MRUB}^{NH}', 'nampismes','')), 
('zz_frac_waste_FEN_DON',(r'\chi_{FLAG\rightarrow MRUB}^{DON}','nampismes','')), 
('zz_frac_waste_FEN_PON',(r'\chi_{FLAG\rightarrow MRUB}^{PON}','nampismes','')), 
('zz_frac_waste_DEZ_NH', (r'\chi_{DIAT\rightarrow MICZ}^{NH}','nampiszoo','')), 
('zz_frac_waste_DEZ_DON',(r'\chi_{DIAT\rightarrow MICZ}^{DON}','nampiszoo','')), 
('zz_frac_waste_DEZ_PON',(r'\chi_{DIAT\rightarrow MICZ}^{PON}','nampiszoo','')), 
('zz_frac_waste_DEZ_Bsi',(r'\chi_{DIAT\rightarrow MICZ}^{bSi}','nampiszoo','')),
('zz_frac_waste_NEZ_NH', (r'\chi_{MRUB\rightarrow MICZ}^{NH}','nampiszoo','')), 
('zz_frac_waste_NEZ_DON',(r'\chi_{MRUB\rightarrow MICZ}^{DON}','nampiszoo','')), 
('zz_frac_waste_NEZ_PON',(r'\chi_{MRUB\rightarrow MICZ}^{PON}','nampiszoo','')), 
('zz_frac_waste_FEZ_NH', (r'\chi_{FLAG\rightarrow MICZ}^{NH}','nampiszoo','')), 
('zz_frac_waste_FEZ_DON',(r'\chi_{FLAG\rightarrow MICZ}^{DON}','nampiszoo','')), 
('zz_frac_waste_FEZ_PON',(r'\chi_{FLAG\rightarrow MICZ}^{PON}','nampiszoo','')), 
('zz_frac_waste_ZEZ_NH', (r'\chi_{MICZ\rightarrow MICZ}^{NH}','nampiszoo','')), 
('zz_frac_waste_ZEZ_DON',(r'\chi_{MICZ\rightarrow MICZ}^{DON}','nampiszoo','')), 
('zz_frac_waste_ZEZ_PON',(r'\chi_{MICZ\rightarrow MICZ}^{PON}','nampiszoo','')), 
('zz_frac_waste_PEZ_NH', (r'\chi_{PON\rightarrow MICZ}^{NH}','nampiszoo','')), 
('zz_frac_waste_PEZ_DON',(r'\chi_{PON\rightarrow MICZ}^{DON}','nampiszoo','')), 
('zz_frac_waste_PEZ_PON',(r'\chi_{PON\rightarrow MICZ}^{PON}','nampiszoo','')), 
('zz_frac_waste_ZNM_NH', (r'\chi_{MICZ\rightarrow MORT}^{NH}','nampiszoo','')), 
('zz_frac_waste_ZNM_DON',(r'\chi_{MICZ\rightarrow MORT}^{DON}','nampiszoo','')), 
('zz_frac_waste_ZNM_PON',(r'\chi_{MICZ\rightarrow MORT}^{PON}','nampiszoo','')), 
('zz_frac_waste_ZEX_NH', (r'\chi_{MICZ\rightarrow EXCR}^{NH}','nampiszoo','')), 
('zz_frac_waste_ZEX_DON',(r'\chi_{MICZ\rightarrow EXCR}^{DON}','nampiszoo','')), 
('zz_frac_waste_ZEX_PON',(r'\chi_{MICZ\rightarrow EXCR}^{PON}','nampiszoo','')), 
('zz_frac_waste_DEM_NH',(r'\chi_{DIAT\rightarrow MESZ}^{NH}','nampismezo','')), 
('zz_frac_waste_DEM_DON',(r'\chi_{DIAT\rightarrow MESZ}^{DON}','nampismezo','')), 
('zz_frac_waste_DEM_PON',(r'\chi_{DIAT\rightarrow MESZ}^{PON}','nampismezo','')), 
('zz_frac_waste_DEM_BSi',(r'\chi_{DIAT\rightarrow MESZ}^{bSi}','nampismezo','')),
('zz_frac_waste_NEM_NH', (r'\chi_{MRUB\rightarrow MESZ}^{NH}','nampismezo','')), 
('zz_frac_waste_NEM_DON',(r'\chi_{MRUB\rightarrow MESZ}^{DON}','nampismezo','')), 
('zz_frac_waste_NEM_PON',(r'\chi_{MRUB\rightarrow MESZ}^{PON}','nampismezo','')), 
('zz_frac_waste_FEM_NH', (r'\chi_{FLAG\rightarrow MESZ}^{NH}','nampismezo','')), 
('zz_frac_waste_FEM_DON',(r'\chi_{FLAG\rightarrow MESZ}^{DON}','nampismezo','')), 
('zz_frac_waste_FEM_PON',(r'\chi_{FLAG\rightarrow MESZ}^{PON}','nampismezo','')), 
('zz_frac_waste_ZEM_NH', (r'\chi_{MICZ\rightarrow MESZ}^{NH}','nampismezo','')), 
('zz_frac_waste_ZEM_DON',(r'\chi_{MICZ\rightarrow MESZ}^{DON}','nampismezo','')), 
('zz_frac_waste_ZEM_PON',(r'\chi_{MICZ\rightarrow MESZ}^{PON}','nampismezo','')), 
('zz_frac_waste_PEM_NH', (r'\chi_{PON\rightarrow MESZ}^{NH}','nampismezo','')), 
('zz_frac_waste_PEM_DON',(r'\chi_{PON\rightarrow MESZ}^{DON}','nampismezo','')), 
('zz_frac_waste_PEM_PON',(r'\chi_{PON\rightarrow MESZ}^{PON}','nampismezo',''))])
#('zz_frac_waste_MNM_NH', (r'\chi_{MESZ\rightarrow MORT}^{NH}','nampismezo','')), 
#('zz_frac_waste_MNM_DON',(r'\chi_{MESZ\rightarrow MORT}^{DON}','nampismezo','')), 
#('zz_frac_waste_MNM_PON',(r'\chi_{MESZ\rightarrow MORT}^{PON}','nampismezo','')), 
#('zz_frac_waste_MEX_NH', (r'\chi_{MESZ\rightarrow EXCR}^{NH}','nampismezo','')), 
#('zz_frac_waste_MEX_DON',(r'\chi_{MESZ\rightarrow EXCR}^{DON}','nampismezo','')), 
#('zz_frac_waste_MEX_PON',(r'\chi_{MESZ\rightarrow EXCR}^{PON}','nampismezo','')),

In [24]:
# make table for paper- all non-zoop
ikey='Photosynthetic Growth'
lines=list()
lines.append(r'   \small'+'\n')
lines.append(r'   \centering'+'\n')
lines.append(r'   \begin{tabular}{lrrl}\toprule'+'\n')
lines.append(r'    Parameter & Current Value & Previous value & Units \\ \midrule'+'\n')
lines.append(r'     \multicolumn{4}{l}{Growth}  \\'+'\n')
for ivar in matchDict[ikey]:
    iltx=matchDict[ikey][ivar][0]
    ival=nml[matchDict[ikey][ivar][1]][ivar]
    ivalold=nmlold[matchDict[ikey][ivar][1]][ivar]
    iunits=matchDict[ikey][ivar][2]
    if not ival==ivalold:
        lines.append(r'     ${}$ & {} & {} & {} \\'.format(iltx,ival,ivalold,iunits)+'\n')
lines.append(r'   \midrule'+'\n')

ikey='Mesodinium rubrum Grazing'
lines.append(r'     \multicolumn{4}{l}{\textit{M. rubrum} Grazing}  \\'+'\n')
for ivar in matchDict[ikey]:
    iltx=matchDict[ikey][ivar][0]
    ival=nml[matchDict[ikey][ivar][1]][ivar]
    ivalold=nmlold[matchDict[ikey][ivar][1]][ivar]
    iunits=matchDict[ikey][ivar][2]
    if not ival==ivalold:
        lines.append(r'     ${}$ & {} & {} & {} \\'.format(iltx,ival,ivalold,iunits)+'\n')
lines.append(r'   \midrule'+'\n')

# make table for paper
ikey='Mortality'
lines.append(r'     \multicolumn{4}{l}{Mortality}  \\'+'\n')
for ivar in matchDict[ikey]:
    iltx=matchDict[ikey][ivar][0]
    ival=nml[matchDict[ikey][ivar][1]][ivar]
    ivalold=nmlold[matchDict[ikey][ivar][1]][ivar]
    iunits=matchDict[ikey][ivar][2]
    if not ival==ivalold:
        lines.append(r'     ${}$ & {} & {} & {} \\'.format(iltx,ival,ivalold,iunits)+'\n')
lines.append(r'   \midrule'+'\n')

# make table for paper
ikey='Remineralization'
lines.append(r'     \multicolumn{4}{l}{Remineralization}  \\'+'\n')
for ivar in matchDict[ikey]:
    iltx=matchDict[ikey][ivar][0]
    ival=nml[matchDict[ikey][ivar][1]][ivar]
    ivalold=nmlold[matchDict[ikey][ivar][1]][ivar]
    iunits=matchDict[ikey][ivar][2]
    if not ival==ivalold:
        lines.append(r'     ${}$ & {} & {} & {} \\'.format(iltx,ival,ivalold,iunits)+'\n')
lines.append(r'   \midrule'+'\n')

# make table for paper
ikey='Light'
lines.append(r'     \multicolumn{4}{l}{Light Attenuation}  \\'+'\n')
for ivar in matchDict[ikey]:
    iltx=matchDict[ikey][ivar][0]
    ival=nml[matchDict[ikey][ivar][1]][ivar]
    ivalold=nmlold[matchDict[ikey][ivar][1]][ivar]
    iunits=matchDict[ikey][ivar][2]
    if not ival==ivalold:
        lines.append(r'     ${}$ & {} & {} & {} \\'.format(iltx,ival,ivalold,iunits)+'\n')
lines.append(r'   \midrule'+'\n')

# make table for paper
ikey='Sinking and Bottom Boundary Condition'
lines.append(r'     \multicolumn{4}{l}{Sinking and Bottom Boundary Condition}  \\'+'\n')
for ivar in matchDict[ikey]:
    iltx=matchDict[ikey][ivar][0]
    ival=nml[matchDict[ikey][ivar][1]][ivar]
    ivalold=nmlold[matchDict[ikey][ivar][1]][ivar]
    iunits=matchDict[ikey][ivar][2]
    if not ival==ivalold:
        lines.append(r'     ${}$ & {} & {} & {} \\'.format(iltx,ival,ivalold,iunits)+'\n')
lines.append(r'   \midrule'+'\n')

## no changes to Detrital Coeffs
#ikey='Detrital Transfer Coefficients'
#lines.append(r'     \multicolumn{4}{l}{Detrital Transfer Coefficients}  \\'+'\n')
#for ivar in matchDict[ikey]:
#    iltx=matchDict[ikey][ivar][0]
#    ival=nml[matchDict[ikey][ivar][1]][ivar]
#    ivalold=nmlold[matchDict[ikey][ivar][1]][ivar]
#    iunits=matchDict[ikey][ivar][2]
#    if not ival==ivalold:
#        lines.append(r'     ${}$ & {} & {} & {} \\'.format(iltx,ival,ivalold,iunits)+'\n')
#lines.append(r'   \midrule'+'\n')
# cut here if add transfer coeffs
lines.append(r' \\ \\')
lines.append(r'   \end{tabular}'+'\n')


f = open('OtherChangedParamsTBL.tex', 'w')
f.writelines(lines)
f.close()